## 1.Load data

In [ ]:
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3\bin;%PATH%
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3\extras\CUPTI\lib64;%PATH%
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3\include;%PATH%
SET PATH=C:\Users\jonat\code\cuda\bin;%PATH%

In [78]:
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
from tensorflow.python.keras.backend import get_session

In [127]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13932601347614081251
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9148379955
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6439351445952173876
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [128]:
import tensorflow as tf
from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto(device_count = {'GPU': 6, 'CPU' : 49} )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)


In [80]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
import pandas as pd
import imblearn
import numpy as np

In [6]:
data = pd.read_csv('./OUTPUTAnnotations/dataset.csv', encoding= 'unicode_escape')
data

,Sentence,Word,Tag
0,Sentence: 1,RECORD,0
1,Sentence: 2,OC,0
2,NaN,AM,0
3,NaN,gallstone,0
4,NaN,pancreatitis,0
...,...,...,...
949802,NaN,M.D.,0
949803,Sentence: 132094,END,0
949804,NaN,OF,0
949805,NaN,DISCHARGE,0


## Groupby

In [7]:
data_fillna = data.fillna(method='ffill', axis=0)
data_group = data_fillna.groupby(['Sentence'],as_index=False
                                )['Word', 'Tag'].agg(lambda x: list(x))

#data_fillna
data_group

C:\Users\jonat\AppData\Local\Temp/ipykernel_11656/794733092.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(['Sentence'],as_index=False


,Sentence,Word,Tag
0,Sentence: 1,[RECORD],[0]
1,Sentence: 10,"[WILL, D/C, ORDER, BE, USED, AS, THE, D/C, SUM...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Sentence: 100,"[prandial, N/V/severe, upper, abdominal, pain....","[0, 1, 0, 1, 1, 0, 0, 0, 0]"
3,Sentence: 1000,"[normal, limits., Cardiac, catheterization, da...","[0, 0, 0, 0, 0, 0, 0, 0]"
4,Sentence: 10000,"[year, old, Black, female, with, significant, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...
132089,Sentence: 99995,"[Height, foot, inch, and, weight, kg., Tempera...","[0, 0, 0, 0, 0, 0, 0]"
132090,Sentence: 99996,"[degrees, heart, rate, and, sinus, blood, pres...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
132091,Sentence: 99997,"[blood, pressure, left, arm, and, oxygen, satu...","[0, 0, 0, 0, 0, 0, 0]"
132092,Sentence: 99998,"[No, carotid, bruits, regular, rate, and, rhyt...","[0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Padding

In [8]:
texts = data_group['Word'].tolist()  
labels = data_group['Tag'].tolist()  

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 34275 unique tokens.


In [10]:
tokensAll = []
tagsAll = []
for tokenLine, tagLine in zip(sequences, labels):
    for token, tag in zip(tokenLine, tagLine):
        tokensAll.append(token)
        tagsAll.append(tag)

In [11]:
len(tokensAll)

949807

In [12]:
len(tagsAll)

949807

In [13]:
indexSequence = 0
tokens607 = []
tags607 = []
count = 0
tokens = []
tags = []
firstTime = 0
appended = 0
while indexSequence < len(tokensAll):
    if count != 128:
        tokens.append(tokensAll[indexSequence])
        tags.append(tagsAll[indexSequence])
        indexSequence = indexSequence + 1
        count = count + 1
    elif count == 128 and firstTime == 0:
        tokens607 = np.array(tokens)
        tags607 = np.array(tags)
        firstTime = 1
        tokens = []
        tags = []
        indexSequence = indexSequence-32
        count = 0 
        appended = 1
        print(appended)
    elif count == 128 and firstTime == 1:
        tokens607 = np.vstack((tokens607, np.array(tokens)))
        tags607 = np.vstack((tags607, np.array(tags)))
        tokens = []
        tags = []
        indexSequence = indexSequence-32
        count = 0 
        appended = appended + 1
        print(appended)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113


3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779
3780
3781
3782
3783
3784
3785
3786
3787


5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447


6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086


8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734


In [14]:
print('Shape of data tensor:', tokens607.shape)
print('Shape of label tensor:', tags607.shape)

Shape of data tensor: (9893, 128)
Shape of label tensor: (9893, 128)


In [15]:
train_tokens, test_tokens, train_tags, test_tags = train_test_split(tokens607, tags607, test_size=0.1, train_size=0.9, random_state=2020)

## BERT Embedding

In [16]:
len(data_group['Word'])

132094

In [17]:
import torch
from transformers import AutoTokenizer, TFAutoModel, AutoModel
from transformers import BertTokenizer, BertTokenizerFast

In [18]:
def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)


def get_hidden_states(encoded, token_ids_word, model, layers):
    with torch.no_grad():
        output = model(**encoded)

    # Get all hidden states
    states = output.hidden_states
    # Stack and sum all requested layers
    output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word]

    return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
    that make up the word of interest, and then `get_hidden_states`."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

    return get_hidden_states(encoded, token_ids_word, model, layers)

In [68]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = BertTokenizerFast.from_pretrained("samrawal/bert-base-uncased_clinical-ner")

model = AutoModelForTokenClassification.from_pretrained("samrawal/bert-base-uncased_clinical-ner", output_hidden_states=True)

In [69]:
layers = [-4, -3, -2, -1]
#tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
#model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

In [70]:
texts1 = data_group['Word'].tolist()  

In [71]:
texts1All = []
for sentence in texts1:
    for word in sentence:
        texts1All.append(word)

In [72]:
len(texts1All)

949807

In [73]:
indexSequence = 0
count = 0
firstTime = 0
sent = ""
added = 0
bert_embedding = ""
while indexSequence < len(texts1All):
    if count != 128:
        word = texts1All[indexSequence]
        sent = sent + word + " "
        finalWord = word
        indexSequence = indexSequence + 1
        count = count + 1
    elif count == 128 and firstTime == 0:
        print(sent)
        print(finalWord)
        idx = get_word_idx(sent, finalWord)
        word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
        bert_embedding = word_embedding
        firstTime = 1
        sent = ""
        final = ""
        idx = 0
        added = added+1
        indexSequence = indexSequence-32
        count = 0 
        print(added)
    elif count == 128 and firstTime == 1:
        idx = get_word_idx(sent, finalWord)
        word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)
        bert_embedding = np.vstack((bert_embedding, word_embedding))
        sent = ""
        final = ""
        idx = 0
        added = added+1
        indexSequence = indexSequence-32
        count = 0 
        print(added)

RECORD WILL D/C ORDER BE USED AS THE D/C SUMMARY: YES prandial N/V/severe upper abdominal pain. Labs were notable for normal limits. Cardiac catheterization data from are as year old Black female with significant past medical history for rales. Soft nontender and nondistended abdomen. Extremities with edema bilaterally. Pulses femoral bilaterally. Pedal pulses are present by Doppler bilaterally. LABORATORY DATA: Admission labs are significant for BUN of creatinine of glucose of white blood cell count of hematocrit of and platelets 000. IMAGING: Cardiac catheterization on performed at Sonprat Health proximal LAD lesion codominant circulation. multiple DVT's who came in here complaining of increased left Echocardiogram on aortic stenosis with mean gradient mmHg peak gradient of mmHg calculated valve area mild aortic insufficiency trivial mitral insufficiency LVH EF of 
of
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884


3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3456
3457
3458
3459
3460
3461
3462
3463
3464
3465
3466
3467
3468
3469
3470
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523


4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4981
4982
4983
4984
4985
4986
4987
4988
4989
4990
4991
4992
4993
4994
4995
4996
4997
4998
4999
5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162


6602
6603
6604
6605
6606
6607
6608
6609
6610
6611
6612
6613
6614
6615
6616
6617
6618
6619
6620
6621
6622
6623
6624
6625
6626
6627
6628
6629
6630
6631
6632
6633
6634
6635
6636
6637
6638
6639
6640
6641
6642
6643
6644
6645
6646
6647
6648
6649
6650
6651
6652
6653
6654
6655
6656
6657
6658
6659
6660
6661
6662
6663
6664
6665
6666
6667
6668
6669
6670
6671
6672
6673
6674
6675
6676
6677
6678
6679
6680
6681
6682
6683
6684
6685
6686
6687
6688
6689
6690
6691
6692
6693
6694
6695
6696
6697
6698
6699
6700
6701
6702
6703
6704
6705
6706
6707
6708
6709
6710
6711
6712
6713
6714
6715
6716
6717
6718
6719
6720
6721
6722
6723
6724
6725
6726
6727
6728
6729
6730
6731
6732
6733
6734
6735
6736
6737
6738
6739
6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6752
6753
6754
6755
6756
6757
6758
6759
6760
6761
6762
6763
6764
6765
6766
6767
6768
6769
6770
6771
6772
6773
6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801


8241
8242
8243
8244
8245
8246
8247
8248
8249
8250
8251
8252
8253
8254
8255
8256
8257
8258
8259
8260
8261
8262
8263
8264
8265
8266
8267
8268
8269
8270
8271
8272
8273
8274
8275
8276
8277
8278
8279
8280
8281
8282
8283
8284
8285
8286
8287
8288
8289
8290
8291
8292
8293
8294
8295
8296
8297
8298
8299
8300
8301
8302
8303
8304
8305
8306
8307
8308
8309
8310
8311
8312
8313
8314
8315
8316
8317
8318
8319
8320
8321
8322
8323
8324
8325
8326
8327
8328
8329
8330
8331
8332
8333
8334
8335
8336
8337
8338
8339
8340
8341
8342
8343
8344
8345
8346
8347
8348
8349
8350
8351
8352
8353
8354
8355
8356
8357
8358
8359
8360
8361
8362
8363
8364
8365
8366
8367
8368
8369
8370
8371
8372
8373
8374
8375
8376
8377
8378
8379
8380
8381
8382
8383
8384
8385
8386
8387
8388
8389
8390
8391
8392
8393
8394
8395
8396
8397
8398
8399
8400
8401
8402
8403
8404
8405
8406
8407
8408
8409
8410
8411
8412
8413
8414
8415
8416
8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440


9880
9881
9882
9883
9884
9885
9886
9887
9888
9889
9890
9891
9892
9893


In [74]:
len(bert_embedding[1])

768

In [75]:
bert_embedding[0]

array([ 9.96012986e-01, -2.04849601e+00,  5.42164278e+00, -1.46302724e+00,
        3.52857852e+00,  4.12632942e+00, -1.53313696e+00, -1.86286402e+00,
       -1.20862436e+00,  2.63902760e+00, -5.97797394e-01,  3.05026197e+00,
        4.57946926e-01, -1.93511701e+00,  3.96743250e+00,  1.49639726e+00,
       -1.02696180e-01,  4.55982637e+00,  2.52709627e+00, -6.41218543e-01,
        4.25721931e+00, -4.40644693e+00,  4.41104555e+00,  1.19929624e+00,
        7.97532499e-02,  1.69055521e+00, -1.34512854e+00,  5.60930431e-01,
       -1.81519330e+00, -1.34857321e+00, -8.53488863e-01,  3.19765162e+00,
        1.73571038e+00,  1.97242379e+00,  1.62174320e+00,  1.30704582e+00,
       -9.28842306e-01, -1.27700162e+00, -3.85558307e-02,  2.85325599e+00,
       -1.23553073e+00, -3.05401158e+00, -3.50195074e+00, -7.61587083e-01,
       -7.01585472e-01,  1.54154420e-01, -2.60981154e+00,  1.52115774e+00,
        3.47226888e-01,  8.61985505e-01, -3.33327389e+00,  1.84846139e+00,
        2.33934760e+00, -

In [76]:
stored_embedding = np.copy(bert_embedding)

## Glove Embedding

In [28]:
import os

embeddings_index = {}
f = open(os.path.join(r'./GLOVE', 'glove.6B.300d.txt'), encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [32]:
len(word_index)

34275

In [33]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [89]:
len(embedding_matrix)

34276

In [47]:
first = 0
bertGloveEmbedding =""
added = 0

for bert, glove in zip(bert_embedding, embedding_matrix):
    print(glove)
    if first == 0:
        bertGloveEmbedding = np.concatenate((bert, glove))
        first = 1
    else:
        embedding = np.concatenate((bert, glove))
        bertGloveEmbedding = np.concatenate((bertGloveEmbedding, embedding), axis = 0)
    added = added+1
    print(added)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1
[ 4.65600006e-02  2.13180006e-01 -7.43639981e-03 -4.58539993e-01
 -3.56389992e-02  2

  2.63520002e-01 -2.88100004e-01  4.43919986e-01  3.28990002e-03]
46
[-0.18559    -0.41771999 -0.019296   -0.065099   -0.41371    -0.035517
 -0.46149001 -0.29153001  0.51858002 -1.54410005 -0.30720001 -0.14391001
  0.30552    -0.22007     0.01946     0.33746001 -0.56525999  0.38698
  0.24942    -0.028965   -0.16752    -0.12778001  0.50777     0.34375
 -0.64033002 -0.39794999 -0.16922     0.40391001 -0.28718999  0.19227999
 -0.3123     -0.18883    -0.36954999  0.074084   -0.77654999 -0.48446
 -0.041167    0.37017     0.14087     0.098732   -0.14069     0.21427999
 -0.066155    0.097496    0.35455     0.30741999  0.29459     0.32901001
 -0.15827     0.070155   -0.016407   -0.18512     0.01506     0.23703
 -0.52591997 -0.58039999  0.66301     0.30177    -0.28255999  0.14469001
 -0.48791     0.31127     0.30807    -0.35299999  0.075481   -0.65902001
 -0.28174999  0.11689    -0.073594   -0.20942     0.010226   -0.39414999
  0.17388999 -0.40566999 -0.24913999  0.27361     0.52613002 -0.30768

 -2.65029997e-01 -5.07380009e-01 -2.04980001e-01 -6.87289983e-02]
89
[-0.30004001  0.53851998 -0.44661999 -0.13113999 -0.0076806   0.25343999
 -0.35464001 -0.1265      0.12999    -1.1997      0.13055     0.03423
 -0.70454001 -0.24353001  0.17973    -0.022131   -0.31966001 -0.24178
 -0.092853   -0.24857    -0.48780999 -0.12433     0.25553     0.14828999
  0.14488    -0.14849    -0.33864999  0.066702   -0.19937    -0.11888
  0.036886    0.39071     0.37990001  0.18010999 -0.70411003  0.11734
  0.40974     0.19244    -0.13095     0.52686     0.25202999 -0.12099
 -0.55387998  0.19241001 -0.080152    0.20926     0.11373     0.31057999
 -0.49623001  0.36662    -0.18464001 -0.56623     0.17171     0.10706
 -0.024185    0.30359    -0.20862     0.16006    -0.42355001 -0.016913
  0.20223001  0.31186     0.11343    -0.44042999  0.13835999 -0.3766
  0.35927999  0.16191    -0.32047001 -0.079465    0.062715    0.012215
 -0.28275001 -0.12834001  0.14675     0.089255   -0.16525    -0.12661
 -0.47204  

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
131
[ 1.46559998e-02  4.53339994e-01 -2.54159987e-01 -9.24040005e-02
 -1.96700007e-01 -3.36530000e-01 -1.81020007e-01 -2.51659989e-01
  4.67510015e-01 -1.82159996e+00 -1.45669997e-01 -3.14619988e-01
 -5.96710026e-01  4.09330018e-02  3.91909987e-01  7.28719980e-02
  4.00669985e-02 -1.50940001e-01 -6.04960024e-01 -2.47580007e-01
 -2.13750005e-01 -4.40710008e-01 -3.70990008e-01  1.76369995e-01
 -5.67550004e-01  4.42580014e-01  2.97129992e-03 -1.48739994e-01
  2.28159994e-01  1.17160000e-01  2.44990006e-01  6.86779976e-01
 -6.00780010e-01 -2.55250007e-01 -5.61479986e-01  4.81680006e-01
  2.82070011e-01  6.27539992e-01  2.92919993e-01 -8.58210027e-02
 -7.22809970e-01 -3.81489992e-01 -1.73879996e-01 -9.78450000e-01
  2.96350002e-01  1.55630007e-01 -1.05180003e-01  4.16099995e-01
  6.59820020e-01 -7.65400007e-02 -1.08840004e-01 -3.13859999e-01
 -3.36609989e-01 -4.37900014e-02  9.86509994e-02  1.46689996e-01
 -1.36050001e-01 -4.01389986e-01 -1.37189999e-02

  2.37839997e-01 -2.95850009e-01  3.27479988e-02  6.02889992e-02]
166
[ 1.43130004e-01 -3.65640000e-02 -7.05630034e-02 -2.12490007e-01
 -3.45899999e-01 -1.67390004e-01 -7.39710033e-02  2.07949996e-01
 -1.08149998e-01 -1.83850002e+00 -2.27660000e-01  6.41139969e-02
 -4.03329998e-01  3.45829993e-01  1.88899994e-01 -4.35090005e-01
 -4.19310004e-01 -1.22950003e-01 -4.86070007e-01 -2.04439998e-01
 -1.87419996e-01 -2.02199996e-01  3.39969993e-01  2.85439998e-01
 -3.02780002e-01  1.30960003e-01 -1.98730007e-01 -2.26960003e-01
 -2.53329992e-01  8.87459964e-02  1.98530003e-01  3.56480002e-01
 -7.26599991e-01  1.89290002e-01 -7.94830024e-01  1.65610000e-01
 -1.87309995e-01  4.26330000e-01  3.63519996e-01  7.15119988e-02
  1.33839995e-01 -4.10710007e-01 -4.30679992e-02  6.06870018e-02
 -2.86900014e-01  1.30339995e-01  2.17760000e-02  5.61649986e-02
 -1.71110004e-01  4.10439998e-01 -9.16939974e-02  1.80099994e-01
 -5.57669997e-02  3.28110009e-02 -1.56670004e-01  2.72550005e-02
 -2.11539999e-01 -1.

...

 -2.73719996e-01 -5.34339994e-02 -2.17600003e-01 -6.81479990e-01]
350
[ 0.56581002  0.29172     0.16526    -0.34909001  0.019348    0.059009
 -0.40176001  0.05946    -0.37720999 -0.35071999 -0.23179001  0.31672001
  0.088153    0.10264     0.32745999 -0.50783002 -0.26440001  0.0084741
 -0.13090999  0.063623   -0.20861     0.027716   -0.027317    1.00629997
  0.12724    -0.049289   -0.53438997  0.14099     0.15350001  0.0744
  0.57072002  0.18854    -1.05620003  0.09748    -0.5984     -0.41106001
  0.32275     0.30153     0.23137    -0.072949    0.065344   -0.35562
  0.25740999  0.65555    -0.27586001  0.29655001 -0.043876    0.34658
 -0.13506     0.19021     0.055573    0.086707    0.22516     0.28759
 -0.13973001  0.20259    -0.37222001 -0.10901     0.58890998  0.21334
  0.39719999  0.1203      0.067338    0.13467     0.56106001 -0.12896
  0.25448     0.44137999 -0.069224    0.12616999 -0.29330999 -0.32480001
 -0.014676   -0.20111001 -0.50098997 -0.23924001  0.23521     0.16455001
 -0

376
[-1.56979993e-01  2.18970001e-01 -1.66260004e-01  1.35130003e-01
 -2.12109998e-01  2.78129995e-01 -1.72969997e-01  9.38249975e-02
  1.69060007e-01 -2.16510010e+00  6.00560009e-02  1.79360006e-02
 -8.80569965e-02 -1.47860005e-01  3.13250005e-01 -2.74790009e-03
 -3.71499985e-01 -2.55690008e-01  4.74370010e-02 -3.24710011e-02
  4.82219994e-01  5.18549979e-01  5.34990013e-01  8.06450024e-02
 -2.91090012e-01 -4.46939990e-02  2.21499994e-01 -1.10919997e-01
 -1.29580006e-01 -1.64399996e-01  8.82679969e-02  4.84169990e-01
 -3.09839994e-01 -6.11670017e-01 -9.58859980e-01  2.77980000e-01
 -3.87649983e-02 -3.21520001e-01 -3.17750007e-01 -1.92919999e-01
 -3.03690005e-02 -3.00069991e-02 -2.67369986e-01 -1.29659995e-01
 -2.51349986e-01 -1.71639994e-01  5.67770004e-02 -1.25390003e-02
 -1.17540002e-01  5.49400002e-02  1.10940002e-01 -4.74609993e-02
  1.62090003e-01  1.68430001e-01 -2.26420000e-01  3.62170011e-01
 -3.30210000e-01  9.12190005e-02  1.07819997e-01  1.70580000e-01
  3.37179989e-01  3.4

 -1.83699995e-01  1.23140000e-01 -5.76120019e-01 -3.23900014e-01]
404
[-0.14065    -0.26594999 -0.020707   -0.26499    -0.36847001 -0.42576
 -0.078387   -0.61748999  0.43325999 -0.027113   -0.64599001  0.42864999
  0.077602   -0.57318997  0.39431     0.22144     0.60971999 -0.30518001
  0.36153999 -0.21814001  0.36409    -0.36138999 -0.093723    0.37261999
 -0.16247    -0.20088001  0.22018    -0.51798999  0.07122     0.16827001
 -0.27102    -0.81207001  0.10475    -0.39243001  0.0077374  -0.18016
 -0.26041001 -0.18703    -0.15429001  0.32251    -0.68826997  0.91368997
  0.084041    0.21087     0.28814    -0.067105    0.60526001  0.41416001
 -0.12667    -0.20582999  0.24593     0.020204    0.63050997 -0.13379
 -0.19848999 -0.31569999  0.056765   -0.34830999  0.45458001  0.55251998
  0.60052001  0.40397    -0.0702      0.50441998  0.58901     0.67359
 -0.14732     0.48708001  0.073362    0.79216999 -0.060468    0.46765
  0.26791999 -0.24517     0.90201998  0.26166001  0.0040899  -0.40327

  0.74712998  0.23097     0.011026   -0.37696001 -0.66769999  0.20774999]
430
[-4.32990015e-01  3.28299999e-01 -9.42749977e-02 -7.45769978e-01
  9.72940028e-02  3.03429991e-01  2.44560003e-01  2.34239995e-01
  1.16439998e-01 -1.38540006e+00 -2.06320003e-01  3.39729995e-01
 -5.39570004e-02  3.14980000e-01  1.14940003e-01  2.92510003e-01
 -2.61839986e-01 -3.13210003e-02  5.10729998e-02 -3.51359993e-01
 -6.87879995e-02  2.79949993e-01  6.61339998e-01  4.90379989e-01
 -4.67810005e-01 -9.00449976e-02 -2.03769997e-01 -3.20920013e-02
 -2.80129999e-01  3.44870001e-01 -1.58769995e-01  4.60249990e-01
 -3.54209989e-01  4.90099996e-01 -3.04080009e-01  4.97000009e-01
 -9.77599993e-02  1.80960000e-01 -7.87350014e-02  4.36189994e-02
 -3.57270017e-02 -5.54720014e-02  5.28649986e-01  3.68470013e-01
  5.80019988e-02 -3.28529999e-02  4.46810007e-01  1.05710000e-01
  2.30000004e-01  5.41840017e-01  4.01089996e-01  2.71319985e-01
  2.60670006e-01  1.64839998e-01 -1.96500003e-01 -6.74820021e-02
 -6.90919995

KeyboardInterrupt: 

In [112]:
np.shape(bert_embedding)

(9893, 768)

In [113]:
np.shape(embedding_matrix)

(34276, 300)

In [124]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import RepeatVector
bertEmbedding_layer = Embedding(9893,
                            768,
                            weights=[bert_embedding],
                            input_length=128,
                            trainable=True)

In [125]:
gloveEmbedding_layer = Embedding(len(word_index) + 1,
                            300,
                            weights=[embedding_matrix],
                            input_length=128,
                            trainable=True)

## Model

In [129]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from keras import backend as K
import keras as keras
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from keras import metrics
from sklearn.utils import class_weight
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [130]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [134]:
def get_bilstm_lstm_model1():
    encoder_input = keras.Input(shape=(128,))
    x = bertEmbedding_layer(encoder_input)
    #x= RepeatVector(len(encoder_input))(x)
    x = keras.layers.Dense(300, activation = "relu")(x)
    y = gloveEmbedding_layer(encoder_input)
    z = keras.layers.Concatenate()([x,y])
    
    modelLayer = keras.layers.Bidirectional(LSTM(units=128, return_sequences=True, recurrent_dropout=0.2), merge_mode = 'concat')(z)
    modelLayer = keras.layers.LSTM(units=128, return_sequences=True, recurrent_dropout=0.5)(modelLayer)
    encoder_output = keras.layers.TimeDistributed(Dense(1, activation="sigmoid"))(modelLayer)
    
    #Optimiser 
    adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
    
    model = keras.Model(encoder_input, encoder_output, name = "model")
    # Compile model
    model.compile(loss='binary_crossentropy', sample_weight_mode="temporal", optimizer='adam', metrics=['acc', precision_m, recall_m, f1_m])
    model.summary()
    
    return model

In [135]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.utils import compute_class_weight
from keras.callbacks import EarlyStopping


skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 999)
skfVal = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 999)

fold_number = 1
f1_per_fold = []
recall_per_fold = []
precision_per_fold = []
acc_per_fold = []
loss_per_fold = []

for i, (trainTotal_index, test_index) in enumerate(skf.split(tokens607, np.sum(tags607, axis = 1))):
    
    print("Training on fold " + str(i+1) + "/10..........")
    
    #Split training set and test set
    x_trainTotal, x_test = tokens607[trainTotal_index], tokens607[test_index]
    y_trainTotal, y_test = tags607[trainTotal_index], tags607[test_index]
    
    for i, (train_index, val_index) in enumerate(skfVal.split(x_trainTotal, np.sum(y_trainTotal, axis = 1))):
        x_train, x_val = x_trainTotal[train_index], x_trainTotal[val_index]
        y_train, y_val = y_trainTotal[train_index], y_trainTotal[val_index]
    
    #Assigning sample weights in training set
    print(str(fold_number) + ": started assigning sample weights")
    weights = class_weight.compute_class_weight(
                                                class_weight ='balanced', 
                                                classes = np.unique(np.ravel(y_train,order='C')), 
                                                y = np.ravel(y_train,order='C')
                                                )
    
    train_tags2 = np.copy(y_train)
    train_tokens2 = np.copy(x_train)
    train_tags2 = train_tags2.astype(float)
    
    indexTotal = 0
    for tags in train_tags2:
        indexTags = 0
        for symptom in tags:
            if symptom == 1:
                train_tags2[indexTotal][indexTags] = float(weights[1]+10.00)
            else:
                train_tags2[indexTotal][indexTags] = float(weights[0])
            indexTags = indexTags+1
        indexTotal = indexTotal + 1
    
    print(str(fold_number) + ": finished assigning sample weights - " + str(weights[0]) + ', ' + str(weights[1] + 10.00))
    weights = train_tags2.reshape((-1, 128, 1))
    print(weights[0])
    #Getting Model Architecture
    model = None 
    model = get_bilstm_lstm_model1()
    
    #Running Model
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
    history = model.fit(x_train, y_train, sample_weight = weights, batch_size=128, verbose=1, epochs=20, validation_data=(x_val, y_val), callbacks=[es])
    
    #Evaluate model
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print(f'Score for fold {fold_number}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}; {model.metrics_names[2]} of {scores[2]}; {model.metrics_names[3]} of {scores[3]}; {model.metrics_names[4]} of {scores[4]} %')
    
    print('-----------------------------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------------------------')

    f1_per_fold.append(scores[4])
    recall_per_fold.append(scores[3])
    precision_per_fold.append(scores[2])
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
    
    #Increase fold number
    fold_number = fold_number + 1

C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Training on fold 1/10..........
1: started assigning sample weights
1: finished assigning sample weights - 0.5113171909139445, 32.59028741328048
[[ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [32.59028741]
 [ 0.51131719]
 [32.59028741]
 [32.59028741]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]
 [ 0.51131719]


C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


2: finished assigning sample weights - 0.5113212054865813, 32.58245405458959
[[ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [32.58245405]
 [ 0.51132121]
 [32.58245405]
 [32.58245405]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.51132121]
 [ 0.511

C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

8: finished assigning sample weights - 0.5113321961323953, 32.56103716166065
[[ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.5113322 ]
 [ 0.511

C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


9: finished assigning sample weights - 0.5113534149968709, 32.51980638150746
[[ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [32.51980638]
 [ 0.51135341]
 [32.51980638]
 [32.51980638]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.51135341]
 [ 0.511

C:\Users\jonat\anaconda3\envs\extraction\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


10: finished assigning sample weights - 0.5113379307869765, 32.54987882684604
[[ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [32.54987883]
 [ 0.51133793]
 [32.54987883]
 [32.54987883]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51133793]
 [ 0.51

In [136]:
#CLINICAL_BERT

print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print("-----------")
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}%')
print('------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------')

Score per fold
-----------
> Fold 1 - Loss: 0.02991165965795517 - Accuracy: 0.9901120662689209 - Precision: 0.7575368285179138 - Recall: 0.8172912001609802 - F1: 0.783405601978302%
-----------
> Fold 2 - Loss: 0.02383757382631302 - Accuracy: 0.9920375347137451 - Precision: 0.7938796877861023 - Recall: 0.8507524728775024 - F1: 0.8193199038505554%
-----------
> Fold 3 - Loss: 0.01922859065234661 - Accuracy: 0.9936947822570801 - Precision: 0.861327052116394 - Recall: 0.8532483577728271 - F1: 0.8552369475364685%
-----------
> Fold 4 - Loss: 0.015432742424309254 - Accuracy: 0.995481550693512 - Precision: 0.8932434320449829 - Recall: 0.89743572473526 - F1: 0.8942782282829285%
-----------
> Fold 5 - Loss: 0.016834665089845657 - Accuracy: 0.9947311282157898 - Precision: 0.9319864511489868 - Recall: 0.8140773773193359 - F1: 0.8679504990577698%
-----------
> Fold 6 - Loss: 0.014522463083267212 - Accuracy: 0.995244562625885 - Precision: 0.9374824166297913 - Recall: 0.8276451826095581 - F1: 0.8779

In [62]:
#BERT_UNCASED

print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print("-----------")
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - Precision: {precision_per_fold[i]} - Recall: {recall_per_fold[i]} - F1: {f1_per_fold[i]}%')
print('------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------')

Score per fold
-----------
> Fold 1 - Loss: 0.027310913428664207 - Accuracy: 0.9921243786811829 - Precision: 0.8035923838615417 - Recall: 0.8164835572242737 - F1: 0.8083021640777588%
-----------
> Fold 2 - Loss: 0.029393043369054794 - Accuracy: 0.9913194179534912 - Precision: 0.7627021670341492 - Recall: 0.8367534875869751 - F1: 0.7963612079620361%
-----------
> Fold 3 - Loss: 0.030222052708268166 - Accuracy: 0.9908538460731506 - Precision: 0.7407005429267883 - Recall: 0.8530195951461792 - F1: 0.7907857298851013%
-----------
> Fold 4 - Loss: 0.027360649779438972 - Accuracy: 0.9918797612190247 - Precision: 0.8183385729789734 - Recall: 0.7700153589248657 - F1: 0.7916640043258667%
-----------
> Fold 5 - Loss: 0.029431860893964767 - Accuracy: 0.9921480417251587 - Precision: 0.8405748009681702 - Recall: 0.760062575340271 - F1: 0.7967877388000488%
-----------
> Fold 6 - Loss: 0.027431242167949677 - Accuracy: 0.9915561676025391 - Precision: 0.7991567850112915 - Recall: 0.7869065403938293 - F1

## Self Check

In [ ]:
predict_tags = model.predict(test_tokens)

In [ ]:
truePositive = 0 
falsePositive = 0
trueNegative = 0 
falseNegative = 0
index = 0
for tag, predTag in zip(test_tags, predict_tags):
    for symptomTag, symptomPred in zip (tag, predTag):
        if symptomPred >= 0.50 and symptomTag == 1:
            truePositive = truePositive + 1
        elif symptomPred >= 0.50 and symptomTag == 0:
            falsePositive = falsePositive + 1
        elif symptomPred < 0.50 and symptomTag == 0:
            trueNegative = trueNegative + 1
        elif symptomPred < 0.50 and symptomTag == 1:
            falseNegative = falseNegative + 1

In [ ]:
print('True postitive: ' + str(truePositive))
print('False postitive: ' + str(falsePositive))
print('True negative: ' + str(trueNegative))
print('False negative: ' + str(falseNegative))

In [ ]:
precision = (truePositive)/(truePositive+falsePositive)
recall = (truePositive)/(truePositive+falseNegative)

In [ ]:
print('Accuracy: ' + str((truePositive+trueNegative)/(truePositive+trueNegative+falsePositive+falseNegative)))
print('Precision: ' + str((truePositive)/(truePositive+falsePositive)))
print('Recall: ' + str((truePositive)/(truePositive+falseNegative)))
print('F1: ' + str((2*precision*recall)/(precision+recall)))

In [ ]:
model.evaluate(pad_tokens, pad_tags)